In [1]:
from pathlib import Path
import os
import wandb
import pandas as pd
import matplotlib.colors as mcolors
import os
import sys
import seaborn as sns
sys.path.append('../')
import numpy as np
import pandas as pd
from advbench.datasets import MNIST, STL10
from advbench.datasets import to_loaders
from advbench.algorithms import ERM, Augmentation, Adversarial_Worst_Of_K, Adversarial_PGD
from advbench.attacks import Fo_Adam
from advbench import hparams_registry
from advbench.lib import  misc
import torch
from advbench.lib.transformations import se_matrix, angle_to_rotation_matrix, se_transform
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
import tarfile
from torch import nn
import shutil

/home/chiche/miniconda3/envs/daug/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
import os
import wandb
import pandas as pd
import matplotlib.colors as mcolors
import os
import sys
import seaborn as sns
sys.path.append('../')
import numpy as np
import pandas as pd
from advbench.datasets import MNIST, STL10
from advbench.datasets import to_loaders
from advbench.algorithms import ERM, Augmentation, Adversarial_Worst_Of_K, Adversarial_PGD
from advbench.attacks import Fo_Adam
from advbench import hparams_registry
from advbench.lib import  misc
import torch
from advbench.lib.transformations import se_matrix, angle_to_rotation_matrix, se_transform
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import nn

import torch
import random
import numpy as np
import os
import json
import pandas as pd
import time
from humanfriendly import format_timespan


from advbench import datasets
from advbench import algorithms
from advbench import attacks
from advbench import hparams_registry
from advbench.lib import misc, meters, plotting, logging
from torch.cuda.amp import autocast
from torchsummary import summary
DEVICE = "cuda"
DOWNLOAD_WEIGHTS = False
PULL_METRICS = False

In [3]:
def load_weights(weight_path, dataset, device=DEVICE):       
        algorithm = ERM(
        dataset.INPUT_SHAPE, 
        dataset.NUM_CLASSES,
        hparams,
        device).to(device)
        w_dict = torch.load(weight_path,map_location=DEVICE)
        #print(weight_dict.keys())
        #if db=="mnist":
        weight_dict = {}
        for s,v in w_dict.items():
                if "classifier" in s and "model" not in s:
                        weight_dict[s.replace('classifier', 'classifier.model')] = v
                else:
                        weight_dict[s] = v 
        algorithm.load_state_dict(weight_dict)
        return algorithm

In [4]:
if PULL_METRICS:
    api = wandb.Api(timeout=50)
    dsets = ["STL10", "MNIST", "CIFAR100"]
    #results_dfs = {}
    entity = "hounie"
    all_runs_list = []
    for dataset in dsets:
        print("Fetching data for {}".format(dataset))
        project = f"DAug-{dataset}"
        base = f"../trained_weights/{project}"
        Path( base ).mkdir( parents=True, exist_ok=True )
        runs = api.runs(f"{entity}/{project}")
        print(f"{len(runs)} runs found")
        for run in runs:
            if run.state == "finished":
                id = run.id
                run = api.run(f"{entity}/{project}/{id}")
                tmp_path = os.path.join(base, f"tmp")
                weight_dir = os.path.join(base, run.name)
                path = os.path.join(weight_dir, f"{run.id}.pkl")
                Path( weight_dir ).mkdir( parents=True, exist_ok=True )
                results = {**run.summary, **run.config, "weights": path, "id": run.id, "name":run.name}
                if DOWNLOAD_WEIGHTS:
                    try:
                        f = run.file("train-output/delta hist_ckpt.pkl").download(tmp_path, replace=True) 
                        os.rename(f.name, path)
                        all_runs_list.append(results)
                    except:
                        try:
                            run = api.run(f"{entity}/{project}/{id}")
                            f = run.file("train-output/loss_ckpt.pkl").download(tmp_path, replace=True)
                            os.rename(f.name, path)
                            all_runs_list.append(results)
                        except:
                            try:
                                run = api.run(f"{entity}/{project}/{id}")
                                f = run.file("train-output/acceptance rate_ckpt.pkl").download(tmp_path, replace=True)
                                os.rename(f.name, path)
                                all_runs_list.append(results)
                            except:
                                print(f"{run.name} {run.id} failed")
                                pass
                else:
                    all_runs_list.append(results)
    df = pd.DataFrame(all_runs_list)
    df.to_csv(f"./results.csv")

Fetching data for STL10
154 runs found
Fetching data for MNIST
212 runs found
Fetching data for CIFAR100
103 runs found


In [5]:
df = pd.read_csv("./results.csv")
test_keys = []
train_keys = []
for c in df.columns:
        if "loss" not in c and "acc" not in c:
            if c.startswith("test"):
                test_keys.append(c)
            else:
                train_keys.append(c)
    

In [6]:
from tqdm import tqdm
data_dir = '../advbench/data'
device = DEVICE
failed = []
#algorithms = ["MH_DALE_PD_Reverse", "Gaussian_DALE_PD_Reverse", "Laplacian_DALE_PD_Reverse", "Adversarial_Smoothed_MH"]
n_aug = {"Rand_Aug":100, "Fo_Adam":10, "Fo_SGD":10}
t_attacks = [k for k in n_aug.keys()]
for exp_id in tqdm(df["id"]):
        try:
            api = wandb.Api(timeout=50)
            exp = df[df["id"]==exp_id]
            dataset =  exp["dataset"].values[0]
            algorithm = "ERM"
            perturbation = exp["perturbation"].values[0]
            model = exp["model"].values[0]
            if True:
                if perturbation == "SE" and "rot" not in model:
                    project = f"DAug-{dataset}"
                    print(dataset, perturbation)
                    t_hparams = exp[test_keys].to_dict(orient='index')[exp.index.values[0]]
                    test_hparams = {}
                    for k, v in t_hparams.items():
                        test_hparams[k.replace("test_","")] = v

                    hparams = exp[train_keys].to_dict(orient='index')[exp.index.values[0]]
                    hparams['model'] = model
                    hparams['epsilon'] = torch.tensor([hparams[f'epsilon_{i}'] for i in ("rot","tx","ty")]).to(device)
                    test_hparams['epsilon'] = hparams['epsilon']
                    hparams['batched'] = False
                    test_hparams['batched'] = False
                    hparams['worst_of_k_steps'] =  100
                    test_hparams['worst_of_k_steps'] =  100
                    aug = hparams["augment"]
                    dataset = vars(datasets)[dataset](data_dir, augmentation = aug)
                    train_ldr, val_ldr, test_ldr = datasets.to_loaders(dataset, hparams, device=device)
                    kw_args = {"perturbation": perturbation}
                    algorithm = load_weights(exp["weights"].values[0], dataset, device=DEVICE)           
                    test_attacks = {a: vars(attacks)[a](algorithm.classifier, test_hparams, DEVICE, perturbation="SE") for a in t_attacks}
                    wandb.init(project=project, resume=exp_id)
                    for attack_name, attack in test_attacks.items():
                        test_adv_acc, test_adv_acc_mean, adv_loss, accs, loss, deltas = misc.adv_accuracy_loss_delta(algorithm, test_ldr, device, attack, augs_per_batch = n_aug[attack_name])
                        train_adv_acc, train_adv_acc_mean, train_adv_loss, train_accs, train_loss, train_deltas = misc.adv_accuracy_loss_delta(algorithm, val_ldr, device, attack, augs_per_batch = n_aug[attack_name])
                        print(f"Logging {attack_name}...")
                        wandb.log({'final test_acc_adv_'+attack_name: test_adv_acc, 'final test_acc_adv_mean'+attack_name: test_adv_acc_mean, 'final test_loss_adv_'+attack_name: adv_loss,
                        'test_loss_adv_mean_'+attack_name: loss.mean()})
                        wandb.log({'final train_acc_adv_'+attack_name: train_adv_acc, 'final train_acc_adv_mean_'+attack_name: train_adv_acc_mean, 'final train_loss_adv_'+attack_name: train_adv_loss,
                        'train_loss_adv_mean_'+attack_name: loss.mean()})
                    wandb.finish(quiet=True)
        except:
            failed.append(exp_id)

with open('failed.txt', 'w') as f:
    for line in failed:
        f.write(f"{line}\n")

  0%|                                                                                                                                | 0/390 [00:00<?, ?it/s]

STL10 SE
Files already downloaded and verified
500
model wrn-16-8-stl
Using WRN-16-8


wandb: Currently logged in as: hounie. Use `wandb login --relogin` to force relogin



0it [00:00, ?it/s]
1it [00:01,  1.39s/it]
2it [00:01,  1.16it/s]
3it [00:02,  1.44it/s]
4it [00:02,  1.62it/s]
5it [00:03,  1.75it/s]
6it [00:03,  1.84it/s]
7it [00:04,  1.91it/s]
8it [00:04,  1.96it/s]
9it [00:05,  1.98it/s]
10it [00:05,  2.01it/s]
11it [00:06,  2.03it/s]
12it [00:06,  2.02it/s]
13it [00:07,  1.99it/s]
14it [00:07,  1.96it/s]
15it [00:08,  1.94it/s]
16it [00:08,  1.93it/s]
17it [00:09,  1.78it/s]
  0%|                                                                                                                                | 0/390 [00:21<?, ?it/s]


KeyboardInterrupt: 

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/home/chiche/miniconda3/envs/daug/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/chiche/miniconda3/envs/daug/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/chiche/miniconda3/envs/daug/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 170, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/home/chiche/miniconda3/envs/daug/lib/python3.9/site-packages/wandb/sdk/interface/interface.py", line 127, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "/home/chiche/miniconda3/envs/daug/lib/python3.9/site-packages/wandb/sdk/interface/interface_shared.py", line 395, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "/home/chiche/miniconda3/envs/daug/lib/python3.9/site-packages/wandb/sdk/interface/interface_shared.py",